In [1]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import random
import re
import builtin.utils as ut

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 3, Finished, Available, Finished)

In [2]:
# Mount external lakehouse  "Bronze" into specified local path.
notebookutils.fs.mount("abfss://8c7d2edc-a6a8-4f1a-bef1-c8b14449cfd3@onelake.dfs.fabric.microsoft.com/a1b6b3d0-9bef-4e2a-a731-81e4d801d7a1", "/lakehouse/Bronze")

# Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/Bronze/Files/Data_Parquet/Video Games Sales.parquet')
df = pd.read_parquet(notebookutils.fs.getMountPath('/lakehouse/Bronze/Files/Data_Parquet/Video Games Sales.parquet'))


StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5ac35f6b-5b26-4624-9633-6d863ac45420)

In [3]:
df.head()

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 5, Finished, Available, Finished)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,NaN,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,None,NaN,None,None
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,NaN,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,None,NaN,None,None


In [4]:
ut.data_summ(df, title=None)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 6, Finished, Available, Finished)

,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,Name,"[<class 'str'>, <class 'NoneType'>]",16717,0.01,2
1,Platform,[<class 'str'>],16719,0.00,0
2,Year_of_Release,[<class 'float'>],16450,1.61,269
3,Genre,"[<class 'str'>, <class 'NoneType'>]",16717,0.01,2
4,Publisher,"[<class 'str'>, <class 'NoneType'>]",16665,0.32,54
5,NA_Sales,[<class 'float'>],15848,5.21,871
6,EU_Sales,[<class 'float'>],15520,7.17,1199
7,JP_Sales,[<class 'float'>],16719,0.00,0
8,Other_Sales,[<class 'float'>],15341,8.24,1378
9,Global_Sales,[<class 'float'>],15728,5.93,991


In [5]:
# Replace the "Role-Playing" values ​​with "RPG" in the "Genre" column.
df['Genre'] = df['Genre'].replace('Role-Playing', 'RPG')
# Change the format of the 'Year_of_Release' column to just the year and type float.
df['Year_of_Release'] = df['Year_of_Release'].apply(lambda x: int(x) if not pd.isna(x) else pd.NA)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 7, Finished, Available, Finished)

In [6]:
# Map the values ​​of the 'Rating' column to the corresponding categories.
# Clear the 'Rating' column of unwanted characters
df['Rating'] = df['Rating'].str.strip()  
df['Rating'] = df['Rating'].replace('', pd.NA)  

# Create the value mapping and consider that the NaNs represent 'RP'
rating_mapping = {
    'EC': 'E',
    'K-A': 'E',
    'E': 'E',
    'E10+': 'E10+',
    'M': 'M',
    'AO': 'AO',
    'T': 'T',
    pd.NA: 'RP',  
    'RP': 'RP'
}

df['Rating'] = df['Rating'].map(rating_mapping)

# Define conditions and new values
conditions = [
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Platform'] == '2600'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Platform'] == 'NES'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Platform'] == 'GB'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Platform'] == 'GBA'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Platform'] == 'GC'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Platform'] == 'GEN'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Platform'] == 'NG'),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'SNES') & (df['Genre'] != 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'SNES') & (df['Genre'] != 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'SNES') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'SNES') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'N64') & (df['Genre'] != 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'N64') & (df['Genre'] != 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'N64') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'N64') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PS') & (df['Genre'] != 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PS') & (df['Genre'] != 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PS') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PS') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'GC') & (df['Genre'] != 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'GC') & (df['Genre'] != 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'GC') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'GC') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'DS') & (df['Genre'] != 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'DS') & (df['Genre'] != 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'DS') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'DS') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == '3DS') & (df['Genre'] != 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == '3DS') & (df['Genre'] != 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == '3DS') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == '3DS') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PSP') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PSP') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PSV') & (df['Genre'] == 'Shooter')),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Platform'] == 'PSV') & (df['Genre'] == 'Fighting')),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Genre'] == 'Racing'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Genre'] == 'Strategy'),
    (df['Rating'].isna() | (df['Rating'] == '')) & (df['Genre'] == 'Platform'),
    (df['Rating'].isna() | (df['Rating'] == '')) & ((df['Genre'] == 'Simulation') & (df['Publisher'] == 'Konami Digital Entertainment'))
]

new_ratindf = [
    'E', 'E', 'E', 'E', 'E', 'E', 'T', 'E', 'E', 'T', 'T', 'E', 'E', 'T', 'T', 'E', 'E', 'T', 'T', 'E10+', 'E10+', 'T', 'T', 'E10+', 'E10+', 'T', 'T', 'E10+', 'E10+', 'T', 'T', 'E10+', 'T', 'E', 'T', 'T', 'T', 'T', 'T'
]

# Use np.select to update the 'Rating' column based on conditions
df['Rating'] = np.select(conditions, new_ratindf, df['Rating'])

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 8, Finished, Available, Finished)

In [7]:
uniquevalues = df['Platform'].unique()
print(uniquevalues)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 9, Finished, Available, Finished)

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']


In [8]:
#Asignar nombres GB, GBA, SAT
platform_mapping = {
    'Wii': 'Nintendo Wii',
    'GEN': 'Sega Genesys',
    'DC': 'Sega Dreamcast',
    'WiiU': 'Nintendo Wii U',
    '3DS': 'Nintendo 3DS',
    'DS': 'Nintendo DS',
    'PS': 'Sony PS',
    'PS2': 'Sony PS2',
    'PS3': 'Sony PS3',
    'PS4': 'Sony PS4',
    'PSV': 'Sony PS Vita',
    'PSP': 'Sony PSP',
    'X360': 'Xbox360',
    'XOne': 'Xbox One'
}

df['Platform'] = df['Platform'].map(platform_mapping).fillna(df['Platform'])

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 10, Finished, Available, Finished)

In [9]:
uniquevalues = df['Platform'].unique()
print(uniquevalues)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 11, Finished, Available, Finished)

['Nintendo Wii' 'NES' 'GB' 'Nintendo DS' 'Xbox360' 'Sony PS3' 'Sony PS2'
 'SNES' 'GBA' 'Sony PS4' 'Nintendo 3DS' 'N64' 'Sony PS' 'XB' 'PC' '2600'
 'Sony PSP' 'Xbox One' 'Nintendo Wii U' 'GC' 'Sega Genesys'
 'Sega Dreamcast' 'Sony PS Vita' 'SAT' 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG'
 'PCFX']


In [10]:
# Defines a dictionary to maps Publishers to Developers
publisher_to_developer = {
    'Capcom': 'Capcom',
    'Disney Interactive Studios': 'Disney Interactive Studios',
    'DreamWorks Interactive': 'Dreamworks',
    'Electronic Arts': 'EA',
    'Electronic Arts Victor': 'EA',
    'Konami Digital Entertainment': 'Konami',
    'LEGO Media': 'TT Games',
    'Mattel Interactive': 'Mattel Games',
    'Microsoft Game Studios': 'Microsoft',
    'Namco Bandai Games': 'Namco',
    'Nintendo': 'Nintendo',
    'Oxigen Interactive': 'Oxigen',
    'Sega': 'Sega',
    'Sony Computer Entertainment': 'Sony Computer Entertainment',
    'SquareSoft': 'Square',
    'Square Enix' : 'Square',
    'Square' : 'Square',
    'Sunsoft': 'Sunsoft',
    'Ubisoft': 'Ubisoft',
    'Ubisoft Annecy': 'Ubisoft'
}

# Define a function to update the Developer
def update_developer(row):
    if pd.isna(row['Developer']) or row['Developer'] == '':
        publisher = row['Publisher']
        if publisher in publisher_to_developer:
            return publisher_to_developer[publisher]
    return row['Developer']

# Apply the function to DataFrame df
df['Developer'] = df.apply(update_developer, axis=1)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 12, Finished, Available, Finished)

In [11]:
df['Developer'].fillna('Unknown', inplace=True)
df['Publisher'].fillna('Unknown', inplace=True)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 13, Finished, Available, Finished)

In [12]:
sales = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales']

sales_not_NA = ['EU_Sales','JP_Sales','Other_Sales']

sales_not_EU = ['NA_Sales','JP_Sales','Other_Sales']

sales_not_other = ['NA_Sales','EU_Sales','JP_Sales']

sales_not_global = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']

mask = df['NA_Sales'].isna() & df[['EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']].notna().all(axis=1)

df.loc[mask, 'NA_Sales'] = df[mask]['Global_Sales'] - df[mask][sales_not_NA].sum(axis = 1)
mask = df['EU_Sales'].isna() & df[['NA_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']].notna().all(axis=1)

df.loc[mask, 'EU_Sales']= df[mask]['Global_Sales'] - df[mask][sales_not_EU].sum(axis = 1)
mask = df['Other_Sales'].isna() & df[['NA_Sales','EU_Sales','JP_Sales','Global_Sales']].notna().all(axis=1)

df.loc[mask, 'Other_Sales']= df[mask]['Global_Sales'] - df[mask][sales_not_other].sum(axis = 1)
mask = df['Global_Sales'].isna() & df[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].isna().any(axis=1)

df[mask]

# Fill empty values ​​in the 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales' columns with 0.
df['NA_Sales'].fillna(0, inplace=True)
df['EU_Sales'].fillna(0, inplace=True)
df['JP_Sales'].fillna(0, inplace=True)
df['Other_Sales'].fillna(0, inplace=True)
df['Global_Sales'].fillna(0, inplace=True)


df['Global_Sales'] = df[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum(axis=1)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 14, Finished, Available, Finished)

In [13]:
uniquevalues = df['Rating'].unique()
print(uniquevalues)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 15, Finished, Available, Finished)

['E' 'E10+' 'M' nan 'T' 'AO' 'RP']


In [14]:
df.dropna(subset=['Name'], inplace=True)
columns_to_drop = ['Critic_Score', 'Critic_Count', 'User_Count']
df = df.drop(columns_to_drop, axis=1)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 16, Finished, Available, Finished)

In [15]:
new_names = {'Name': 'Item_name', 'Year_of_Release': 'Release', 'Genre': 'Genres', 'average_playtime': 'Average_playtime', 'User_Score': 'Score'}

# Use the rename() method to change column names
df.rename(columns=new_names, inplace=True)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 17, Finished, Available, Finished)

In [16]:
# Crear 'Item_id' con un valor predeterminado
df['Item_id'] = None  # O usa cualquier valor predeterminado, como 0 o ''

column_order = ['Item_id', 'Item_name', 'Platform', 'Genres', 'Release', 'Developer', 'Publisher',  'Rating', 'Score', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
gs = df[column_order]

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 18, Finished, Available, Finished)

In [17]:
gs

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 19, Finished, Available, Finished)

,Item_id,Item_name,Platform,Genres,Release,Developer,Publisher,Rating,Score,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,None,Wii Sports,Nintendo Wii,Sports,2006,Nintendo,Nintendo,E,8,41.36,28.96,3.77,8.44,82.53
1,None,Super Mario Bros.,NES,Platform,1985,Nintendo,Nintendo,E,None,29.08,3.58,6.81,0.77,40.24
2,None,Mario Kart Wii,Nintendo Wii,Racing,2008,Nintendo,Nintendo,E,8.3,15.68,12.76,3.79,3.29,35.52
3,None,Wii Sports Resort,Nintendo Wii,Sports,2009,Nintendo,Nintendo,E,8,15.61,10.93,3.28,2.95,32.77
4,None,Pokemon Red/Pokemon Blue,GB,RPG,1996,Nintendo,Nintendo,E,None,11.27,8.89,10.22,1.00,31.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,None,Samurai Warriors: Sanada Maru,Sony PS3,Action,2016,Unknown,Tecmo Koei,NaN,None,0.00,0.00,0.01,0.00,0.01
16715,None,LMA Manager 2007,Xbox360,Sports,2006,Unknown,Codemasters,NaN,None,0.00,0.01,0.00,0.00,0.01
16716,None,Haitaka no Psychedelica,Sony PS Vita,Adventure,2016,Unknown,Idea Factory,NaN,None,0.00,0.00,0.01,0.00,0.01
16717,None,Spirits & Spells,GBA,Platform,2003,Unknown,Wanadoo,E,None,0.01,0.00,0.00,0.00,0.01


In [18]:
archivo_parquet = '/lakehouse/default/Files/Cleaned_Data/games_console.parquet'

# Export to a parquet file.
gs.to_parquet(archivo_parquet, index=False)

StatementMeta(, 7dfb9b56-0ba1-4919-92e8-a20def44cf70, 20, Finished, Available, Finished)